# BYOC Spacy Pre-Trained NER Serverless Endpoint Deployment

[Container Structure](https://sagemaker-workshop.com/custom/containers.html)
- NER
    - predictor.py: (Flask app for inference)
    - wsgi.py: (Wrapper around predictor)
    - nginx.conf: (Config for nginx front-end)
    - serve: program for container hosting, launches gunicorn server
    - Note that there is no train for pre-trained
- Dockerfile

# Push Docker Image to ECR

In [1]:
%%sh

# Name of algo -> ECR
algorithm_name=sm-pretrained-spacy

cd container

#make serve executable
chmod +x NER/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Region, defaults to us-west-2
region=$(aws configure get region)
region=${region:-us-east-1} #set your region here

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Sending build context to Docker daemon  10.24kB
Step 1/10 : FROM python:3.8
3.8: Pulling from library/python
0e29546d541c: Pulling fs layer
9b829c73b52b: Pulling fs layer
cb5b7ae36172: Pulling fs layer
6494e4811622: Pulling fs layer
6f9f74896dfa: Pulling fs layer
fcb6d5f7c986: Pulling fs layer
35b0d149a82c: Pulling fs layer
700a07047b6b: Pulling fs layer
793b1b0c3ddf: Pulling fs layer
fcb6d5f7c986: Waiting
35b0d149a82c: Waiting
700a07047b6b: Waiting
793b1b0c3ddf: Waiting
6494e4811622: Waiting
6f9f74896dfa: Waiting
9b829c73b52b: Verifying Checksum
9b829c73b52b: Download complete
cb5b7ae36172: Verifying Checksum
cb5b7ae36172: Download complete
0e29546d541c: Verifying Checksum
0e29546d541c: Download complete
6494e4811622: Verifying Checksum
6494e4811622: Download complete
fcb6d5f7c986: Verifying Checksum
fcb6d5f7c986: Download complete
700a07047b6b: Verifying Checksum
700a07047b6b: Download complete
35b0d149a82c: Verifying Checksum
35b0d149a82c: Download complete
793b1b0c3

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



# SageMaker Client Setup

- [SageMaker Client](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_model): Model, Endpoint Config, and Endpoint Creation
- [SageMaker RunTime Client](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker-runtime.html#SageMakerRuntime.Client.invoke_endpoint): Endpoint Invocation/Testing

In [2]:
import boto3
from sagemaker import get_execution_role

sm_client = boto3.client(service_name='sagemaker')
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')

account_id = boto3.client('sts').get_caller_identity()['Account']
region = boto3.Session().region_name

#not really used in this use case, use when need to store model artifacts (Ex: MME)
s3_bucket = 'spacy-sagemaker-us-east-1-bucket'

role = get_execution_role()

# Model Creation
[Documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_model)

In [3]:
from time import gmtime, strftime

model_name = 'spacy-nermodel-serverless' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = 's3://{}/spacy/'.format(s3_bucket) ## MODEL S3 URL
container = '{}.dkr.ecr.{}.amazonaws.com/sm-pretrained-spacy:latest'.format(account_id, region)
instance_type = 'ml.c5d.18xlarge'

print('Model name: ' + model_name)
print('Model data Url: ' + model_url)
print('Container image: ' + container)

container = {
    'Image': container
}

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    Containers = [container])

print("Model Arn: " + create_model_response['ModelArn'])

Model name: spacy-nermodel-serverless2022-01-08-21-55-06
Model data Url: s3://spacy-sagemaker-us-east-1-bucket/spacy/
Container image: 474422712127.dkr.ecr.us-east-1.amazonaws.com/sm-pretrained-spacy:latest
Model Arn: arn:aws:sagemaker:us-east-1:474422712127:model/spacy-nermodel-serverless2022-01-08-21-55-06


# Endpoint Config Creation
Adjust Serverless Config here: [Documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint_config)

In [4]:
endpoint_config_name = 'spacy-ner-config-serverless' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint config name: ' + endpoint_config_name)

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "byoVariant",
            "ModelName": model_name,
            "ServerlessConfig": {
                "MemorySizeInMB": 4096,
                "MaxConcurrency": 1,
            },
        },
    ],
)

print("Endpoint config Arn: " + endpoint_config_response['EndpointConfigArn'])

Endpoint config name: spacy-ner-config-serverless2022-01-08-21-56-26
Endpoint config Arn: arn:aws:sagemaker:us-east-1:474422712127:endpoint-config/spacy-ner-config-serverless2022-01-08-21-56-26


# Endpoint Creation
[Documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint)

In [5]:
%%time

import time

endpoint_name = 'spacy-ner-endpoint-serverless' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint name: ' + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Endpoint Status: " + status)

print('Waiting for {} endpoint to be in service...'.format(endpoint_name))
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

Endpoint name: spacy-ner-endpoint-serverless2022-01-08-21-56-34
Endpoint Arn: arn:aws:sagemaker:us-east-1:474422712127:endpoint/spacy-ner-endpoint-serverless2022-01-08-21-56-34
Endpoint Status: Creating
Waiting for spacy-ner-endpoint-serverless2022-01-08-21-56-34 endpoint to be in service...
CPU times: user 97.7 ms, sys: 9.38 ms, total: 107 ms
Wall time: 4min 2s


# Endpoint Invocation

[Documentation](https://boto3.amazonaws.com/v1/documentation/api/1.9.42/reference/services/sagemaker-runtime.html#SageMakerRuntime.Client.invoke_endpoint)

In [7]:
import json
content_type = "application/json"
request_body = {"input": "This is a test with NER in America with Amazon and Microsoft in Seattle, writing random stuff."}

#Serialize data for endpoint
data = json.loads(json.dumps(request_body))
payload = json.dumps(data)

#Endpoint invocation
response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=content_type,
    Body=payload)

#Parse results
result = json.loads(response['Body'].read().decode())['output']
result

[['NER', 'ORG'],
 ['America', 'GPE'],
 ['Amazon', 'ORG'],
 ['Microsoft', 'ORG'],
 ['Seattle', 'GPE']]